## READ DATA

In [35]:
# Read the data text file
with open ("./Dataset/sherlock-holm.es_stories_plain-text_advs.txt", 'r') as file:
    text = file.read()

## TOKENIZE THE TEXT 

In [36]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Initialize the tokenizer
tokenizer = Tokenizer()
# Fit the tokenizer on the text to create a vocabulary
tokenizer.fit_on_texts([text])
# Get the total number of unique words
total_words = len(tokenizer.word_index) + 1

In [37]:
total_words

8200

## FORMING N-GRAMS

In [38]:
input_sequences = []  # Initialize an empty list to store input sequences

# Split the text into lines using the newline character as a delimiter
for line in text.split('\n'):  
    # Convert the line of text into a sequence of tokens based on the tokenizer's vocabulary
    token_list = tokenizer.texts_to_sequences([line])[0]  

    # Iterate over the token list to create n-gram sequences
    for i in range(1, len(token_list)):  
        # Extract a subsequence (n-gram) from the beginning of the token list up to the current index i+1
        n_gram_sequence = token_list[:i+1]  
        # Append the n-gram sequence to the list of input sequences
        input_sequences.append(n_gram_sequence)  

## PAD THE INPUT SEQUENCES TO HAVE EQUAL LENGHT

In [39]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Calculate the length of the longest sequence in the input_sequences list
max_sequence_len = max([len(seq) for seq in input_sequences])

# Pad all sequences in input_sequences to have the same length as the longest sequence
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

## SPLIT THE SEQUENCES INTO INPUT AND OUTPUT

In [40]:
# Assign X the input sequences without the last token in each sequence
X = input_sequences[:, :-1]
# Assign y the last token of each sequence as the target output
y = input_sequences[:, -1]

- **[:, :-1]:** This notation means "select all rows, and all columns except the last one.
- **[:, -1]:** This notation means "select all rows, and only the last column.

## CONVERT THE OUTPUT TO ONE-HOT ENCODE VECTORS 

In [41]:
import tensorflow as tf

# Convert the output array 'y' to one-hot encoded vectors
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

## BUILD THE NEURAL NETWORK ARCHI 

In [43]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense


model = Sequential(
    [
        Embedding(total_words, 100, input_length=max_sequence_len-1),
        LSTM(150),
        Dense(total_words, activation='softmax'),
    ]
)
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2,208,800
Trainable params: 2,208,800
Non-trainable params: 0
_________________________________________________________________
None


## COMPILE AND TRAIN THE MODEL

In [44]:
# Compile the model : prepares the model for training
model.compile(
    loss='categorical_crossentropy', # Loss function to measure how well the model is performing
    optimizer='adam',                # Optimizer to update the model weights during training
    metrics=['accuracy']             # Metric to evaluate the model's performance
)

# Train the model with the training data
model.fit(
    X,           # Input data (features)
    y,           # Target data (labels)
    epochs=100,  # Number of times to iterate over the entire dataset
    verbose=1    # Verbosity mode, 1 means progress bar with logs for each epoch
)


Epoch 1/100
3010/3010 [==============================] - 152s 50ms/step - loss: 6.2248 - accuracy: 0.0780
Epoch 2/100
3010/3010 [==============================] - 126s 42ms/step - loss: 5.4917 - accuracy: 0.1264
Epoch 3/100
3010/3010 [==============================] - 138s 46ms/step - loss: 5.1127 - accuracy: 0.1484
Epoch 4/100
3010/3010 [==============================] - 126s 42ms/step - loss: 4.7900 - accuracy: 0.1669
Epoch 5/100
3010/3010 [==============================] - 129s 43ms/step - loss: 4.4891 - accuracy: 0.1830
Epoch 6/100
3010/3010 [==============================] - 101s 34ms/step - loss: 4.2032 - accuracy: 0.2034
Epoch 7/100
3010/3010 [==============================] - 104s 35ms/step - loss: 3.9339 - accuracy: 0.2268
Epoch 8/100
3010/3010 [==============================] - 104s 35ms/step - loss: 3.6804 - accuracy: 0.2570
Epoch 9/100
3010/3010 [==============================] - 107s 35ms/step - loss: 3.4447 - accuracy: 0.2886
Epoch 10/100
3010/3010 [======================

3010/3010 [==============================] - 102s 34ms/step - loss: 0.5363 - accuracy: 0.8643
Epoch 78/100
3010/3010 [==============================] - 105s 35ms/step - loss: 0.5311 - accuracy: 0.8649
Epoch 79/100
3010/3010 [==============================] - 103s 34ms/step - loss: 0.5338 - accuracy: 0.8630
Epoch 80/100
3010/3010 [==============================] - 102s 34ms/step - loss: 0.5292 - accuracy: 0.8649
Epoch 81/100
3010/3010 [==============================] - 102s 34ms/step - loss: 0.5335 - accuracy: 0.8639
Epoch 82/100
3010/3010 [==============================] - 103s 34ms/step - loss: 0.5261 - accuracy: 0.8649
Epoch 83/100
3010/3010 [==============================] - 103s 34ms/step - loss: 0.5249 - accuracy: 0.8648
Epoch 84/100
3010/3010 [==============================] - 103s 34ms/step - loss: 0.5223 - accuracy: 0.8672
Epoch 85/100
3010/3010 [==============================] - 102s 34ms/step - loss: 0.5196 - accuracy: 0.8669
Epoch 86/100
3010/3010 [==========================

In [59]:
# Initial seed text to start the prediction
seed_text = "i am"

# Number of words to predict
next_words = 3

# Loop to generate the next words
for _ in range(next_words):
    # Convert the seed text into a sequence of tokens
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    
    # Pad the sequence to match the input length of the model
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    
    # Predict the next word's index
    predicted = np.argmax(model.predict(token_list), axis=-1)
    
    # Find the word that corresponds to the predicted index
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    
    # Add the predicted word to the seed text
    seed_text += " " + output_word

# Print the final generated text
print(seed_text)


1/1 [==============================] - 0s 23ms/step
i am glad to hear
